## Name: Luke Pratley
## Date Week 6

The purpose of this notebook is to see if the road dataset has overlap with the building dataset. Then we can create road masks that compliment the building foot print mask. The road dataset covers more area, so we expect only a subset of roads to be present in both datasets.

In [55]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import os

import geopandas as gpd
import rasterio
import rasterio.features
from rasterio.merge import merge
from rasterio.plot import show


from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [56]:
data_directory = 'D:\Capstone Project\data\\'

In [57]:
datasets = glob.glob(data_directory + '*\\')
datasets = [d.split('\\')[-2] for d in datasets]
datasets

['AOI_2_Vegas',
 'AOI_2_Vegas_Train',
 'AOI_3_Paris',
 'AOI_3_Paris_Train',
 'AOI_4_Shanghai',
 'AOI_4_Shanghai_Train',
 'AOI_5_Khartoum',
 'AOI_5_Khartoum_Train']

Below I am listing the directories in the dataset. 

In [81]:
directories = glob.glob(data_directory + f'{datasets[6]}\*')
directories.sort()

directories

['D:\\Capstone Project\\data\\AOI_5_Khartoum\\MS',
 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\PAN',
 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\PS-MS',
 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\PS-RGB',
 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\geojson_roads']

This dataset doesn't have a summary table like the buildings one did.

## Data Cleaning

Here we are reading in information about the labels; the table containing the building locations:

In [82]:
geojson_files = [gpd.read_file(k) for k in glob.glob(directories[-1]+'\*')]

C:\Users\lukep\miniconda3\envs\tf-gpu\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)
C:\Users\lukep\miniconda3\envs\tf-gpu\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)
C:\Users\lukep\miniconda3\envs\tf-gpu\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)
C:\Users\lukep\miniconda3\envs\tf-gpu\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the 

We can take a quick look at a file:

In [83]:
geojson_files[1]

,bridge_type,heading,lane_numbe,lane_number,one_way_type,paved,road_id,road_type,origarea,origlen,partialDec,truncated,geometry
0,2,0,2,2,2,1,7368,4,0,0.000397,1,0,"LINESTRING (32.53411 15.63126, 32.53415 15.63165)"
1,2,0,1,1,2,2,8213,5,0,0.000365,1,0,"LINESTRING (32.53343 15.63132, 32.53345 15.63168)"
2,2,0,2,2,2,2,10384,4,0,0.000360,1,0,"LINESTRING (32.53281 15.63137, 32.53283 15.631..."
3,2,0,2,2,2,2,10483,5,0,0.000700,1,0,"LINESTRING (32.53415 15.63165, 32.53345 15.63168)"
4,2,0,2,2,2,2,10737,5,0,0.000602,1,0,"LINESTRING (32.53345 15.63168, 32.53311 15.631..."
5,2,0,2,2,2,1,5146,4,0,0.000396,1,0,"LINESTRING (32.53417 15.63205, 32.53415 15.63165)"
6,2,0,2,2,2,2,6114,5,0,0.000405,1,0,"LINESTRING (32.53347 15.63208, 32.53345 15.63168)"
7,2,0,2,2,2,2,8151,4,0,0.000429,1,0,"LINESTRING (32.53289 15.63215, 32.53286 15.631..."
8,2,0,2,2,2,2,6376,5,0,0.000665,1,0,"LINESTRING (32.53417 15.63205, 32.53351 15.63208)"
9,2,0,2,2,2,2,8429,5,0,0.000036,1,0,"LINESTRING (32.53351 15.63208, 32.53347 15.63208)"


We can see information about the types of roads!

Below we concatenate all of the files:

In [84]:
road_df = pd.concat(geojson_files, axis=0, ignore_index=True)

In [85]:
road_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10103 entries, 0 to 10102
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   bridge_type   10103 non-null  object  
 1   heading       10103 non-null  object  
 2   lane_numbe    10103 non-null  object  
 3   lane_number   10103 non-null  object  
 4   one_way_type  10103 non-null  object  
 5   paved         10103 non-null  object  
 6   road_id       10103 non-null  int64   
 7   road_type     10103 non-null  object  
 8   origarea      10103 non-null  int64   
 9   origlen       10103 non-null  float64 
 10  partialDec    10103 non-null  int64   
 11  truncated     10103 non-null  int64   
 12  geometry      10103 non-null  geometry
dtypes: float64(1), geometry(1), int64(4), object(7)
memory usage: 1.0+ MB


There are no nulls in any of the columns. We find that there are duplicates for the road Ids.

In [86]:
road_df['road_id'].duplicated().sum()

1134

In [87]:
len(road_df['road_id'].unique())

8969

There are 16k roads

There is a non-truncated version of each of these roads.

In [88]:
road_df['road_id'].shape[0]

10103

Solving for the unique road gemoetry rows and keeping only the first:

In [89]:
unique_roads = road_df['road_id']

In [90]:
unique_roads

0         8013
1        10836
2        11989
3         6562
4         6731
         ...  
10098     3253
10099      106
10100    12681
10101     1692
10102    10624
Name: road_id, Length: 10103, dtype: int64

In [91]:
final_road_df = road_df.iloc[unique_roads.index, :]

In [92]:
final_road_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 10103 entries, 0 to 10102
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   bridge_type   10103 non-null  object  
 1   heading       10103 non-null  object  
 2   lane_numbe    10103 non-null  object  
 3   lane_number   10103 non-null  object  
 4   one_way_type  10103 non-null  object  
 5   paved         10103 non-null  object  
 6   road_id       10103 non-null  int64   
 7   road_type     10103 non-null  object  
 8   origarea      10103 non-null  int64   
 9   origlen       10103 non-null  float64 
 10  partialDec    10103 non-null  int64   
 11  truncated     10103 non-null  int64   
 12  geometry      10103 non-null  geometry
dtypes: float64(1), geometry(1), int64(4), object(7)
memory usage: 1.1+ MB


In [93]:
print(final_road_df['geometry'][0])

LINESTRING (32.48751129243152 15.517321199936934, 32.487438558 15.517031424)


It turns out this is enough to create a mask after reading an image from the building dataset, if we use the functions below

In [97]:

image_names = np.load(datasets[7] +'_image_names.npy')

For example:

In [98]:
image_names[:10]

array(['AOI_5_Khartoum_img1', 'AOI_5_Khartoum_img10',
       'AOI_5_Khartoum_img100', 'AOI_5_Khartoum_img1001',
       'AOI_5_Khartoum_img1003', 'AOI_5_Khartoum_img1004',
       'AOI_5_Khartoum_img1007', 'AOI_5_Khartoum_img1008',
       'AOI_5_Khartoum_img1010', 'AOI_5_Khartoum_img1011'], dtype='<U22')

In [99]:
import time

count  = 0
zero_count = 0
for k, im in enumerate(image_names):
    im_id = im
    path = f'D:\\Capstone Project\\Data\\{datasets[7]}\\road_mask\\road_mask_{im_id}.npy'
    assert not os.path.exists(path), "Mask already exists!"
    with open(path, 'wb') as f:
        with rasterio.open(f'D:\\Capstone Project\\data\\{datasets[7]}\\RGB-PanSharpen\\RGB-PanSharpen_' + im + '.tif') as image:
            mask = rasterio.features.geometry_mask(final_road_df['geometry'], image.shape, image.transform, all_touched=False, invert=True)
            if mask.sum() == 0:
                zero_count += 1
                print(zero_count)
        count += 1
        np.save(f, mask)
        if k % 500 == 0:
            print(k, im)


0 AOI_5_Khartoum_img1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
500 AOI_5_Khartoum_img282
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
